In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv

In [3]:
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By

In [31]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

: 

In [4]:
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

Fase 2: Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizaras la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas (IMDB y Rotten Tomatoes) y extraer detalles como calificaciones, actores y directores. En concreto deberás extraer información sobre:

Puntuacion de IMDB (en caso de que la tenga).

Puntuación de Rotten Tomatoes (Tomatometer).

Dirección (director/a o directore/as de cada película).

Gionistas (de cada película).

Argumento.

Duración (en minutos).

Nombre de la película

Probar un try para el buscador y en except break, asi me devuelve los resultados en csv hasta el momento?

In [25]:
# # Llamo al navegador

# driver=webdriver.Chrome()
# driver.get("https://www.imdb.com/?ref_=nv_home")
# driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# # Hago grande el navegador para que aparezcan todas las opciones posibles de la web

# driver.maximize_window()

# sleep(5)


def llamar_info_peli(lista_id_muestra,driver,archivo_csv):
  # Creo lista de tuplas tal y como pide el enunciado para guardar los datos extraidos 

  lista_tuplas_pelis=[]

  # Creamos una muestras de id de peliculas por las que vamos a iterar para extraer la info

  #lista_id_muestra= ['tt10579986','tt10597228','tt11097384','tt11152168','tt11237950']


  for id in lista_id_muestra:
      
      # Creo la lista donde voy a ir guardando la info de cada peli, que despues convertiremos en tupla 

      lista_datos_imdb=[]

      try: 

        sleep(5)

        busqueda = driver.find_element("css selector","#suggestion-search")

        busqueda.clear()

        busqueda.send_keys(id, Keys.ENTER)

        # Extraemos la informacion que necesitamos

        nombre_pelicula=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/span').text
        #nombre_pelicula=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > h1 > span").text 
      
        lista_datos_imdb.append(nombre_pelicula)


        # DIRECCION: extraemos la informacion de los directores, que pueden ser 1 o varios
        # Creamos una lista donde iremos agregando los diferentes directores en base a las etiquetas correspondientes

        #direccion_peli=[]

        try: 
          
          director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[1]/div/ul/li[1]/a').text
          lista_datos_imdb.append(director1) 

        except:  

            try:
                   
              director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[1]/div/ul/li/a').text
              lista_datos_imdb.append(director1)

            except:
               
               director2= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]/div').text
               lista_datos_imdb.append(director2)     
                                            
        # Guionistas

        try:
          
          guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[2]/div').text
          guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
          lista_datos_imdb.append(guionistas_bien)

        except:
            try:
              guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[2]/div').text
              guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
              lista_datos_imdb.append(guionistas_bien)

            except:
                guionistas_peli=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[2]/div').text
                guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
                lista_datos_imdb.append(guionistas_bien) 


        # Argumento
        # Para argumento hay varias etiquetas, una para cada tamaño de pantalla o dispositivo, por lo que hay que escoger la de pantalla grande
        
        argumento_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
        lista_datos_imdb.append(argumento_peli) 

      
        # Duracion pelicula:

        try: 
          duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]').text
          #duracion_peli= driver.find_element("css selector","__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(28) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(1) > div").text
          lista_datos_imdb.append(duracion_peli)

        except:

          try:  
            duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]').text
            lista_datos_imdb.append(duracion_peli)
            
          except:
            lista_datos_imdb.append("No encontrado")
          
    
        # Puntuacion
        try:
          
          puntuacion_imdb= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
          lista_datos_imdb.append(puntuacion_imdb.replace(",", "."))

        except:
        
          lista_datos_imdb.append("No encontrado")

      except Exception as null:
          print(f"Error al procesar la búsqueda para el ID {id}: {str(null)}")

    
    # Guardo todos los datos extraidos de cada peli en una tupla, que se incluye, a su vez, en la lista de tuplas de todas las peliculas

      lista_tuplas_pelis.append(tuple(lista_datos_imdb))

  # Guardo informacion en un csv

  with open(archivo_csv, 'w', newline='', encoding='utf-8') as archivo:
         
     escritor =csv.writer(archivo) 
     escritor.writerow(["Nombre", "Director", "Guionista", "Argumento", "Duración", "Puntuación IMDb"])
     escritor.writerows(lista_tuplas_pelis)
  
  # Sale a pagina principal y vuelve a empezar bucle

  driver.back()

  return lista_tuplas_pelis

In [26]:
#lista_tuplas_pelis

[("Mom's Coming",
  'Rishiking',
  'Rishiking',
  'Un alma supuestamente tiene como hija a una chica de 20 años, a la que quiere llevarse con ella, y la hermana mayor quiere impedir que el alma lo haga.',
  '2h 2min',
  'No encontrado'),
 ('Love Song and Power',
  'Erik Krefeld',
  'Stevie Kinchen Erik Krefeld',
  'Esta es una historia sobre encontrar el valor para ser uno mismo.',
  '1h 30min',
  'No encontrado'),
 ('El astronauta',
  'Johan Renck',
  'Jaroslav Kalfar Colby Day',
  'Jakub Procházka, huérfano y criado en el campo por sus abuelos, supera todas las adversidades y se convierte en el primer astronauta del país.',
  '1h 47min',
  '5.7'),
 ('Amigos imaginarios',
  'John Krasinski',
  'John Krasinski',
  'Sigue a una niña que pasa por una experiencia difícil y entonces empieza a ver a los amigos imaginarios de todo el mundo que se han quedado atrás cuando sus amigos de la vida real han crecido.',
  '1h 44min',
  '8.0'),
 ('The Accompanist Awakening',
  'Frederick Keeve',
  'F

In [6]:
#resultados_muestra

[("Mom's Coming",
  'Rishiking',
  'Rishiking',
  'Un alma supuestamente tiene como hija a una chica de 20 años, a la que quiere llevarse con ella, y la hermana mayor quiere impedir que el alma lo haga.',
  '2h 2min',
  'No encontrado'),
 ('Love Song and Power',
  'Erik Krefeld',
  'Stevie Kinchen Erik Krefeld',
  'Esta es una historia sobre encontrar el valor para ser uno mismo.',
  '1h 30min',
  'No encontrado'),
 ('El astronauta',
  'Johan Renck',
  'Jaroslav Kalfar Colby Day',
  'Jakub Procházka, huérfano y criado en el campo por sus abuelos, supera todas las adversidades y se convierte en el primer astronauta del país.',
  '1h 47min',
  '5.7'),
 ('Amigos imaginarios',
  'John Krasinski',
  'John Krasinski',
  'Sigue a una niña que pasa por una experiencia difícil y entonces empieza a ver a los amigos imaginarios de todo el mundo que se han quedado atrás cuando sus amigos de la vida real han crecido.',
  '1h 44min',
  '7.6'),
 ('The Accompanist Awakening',
  'Frederick Keeve',
  'F

In [5]:
# Importamos el csv de Accion para extraer los ID y llamar a la funcion

datos_accion_csv=pd.read_csv(r'C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_Acción_2010-2025.csv')

In [6]:
# Quiero ver como se llaman las columnas del csv para buscar la columna de ID

print(datos_accion_csv.columns)

Index(['Género', 'Tipo', 'Título', 'Mes', 'Año', 'ID_IMDB'], dtype='object')


In [7]:
# Extraemos los id y posteriormente los convertiremos a una lista

id_accion=datos_accion_csv["ID_IMDB"]
id_accion

0        tt0411951
1        tt0423474
2        tt0429493
3        tt0439801
4        tt0446029
           ...    
5236    tt17506650
5237    tt18072316
5238    tt18340690
5239    tt19892640
5240    tt20873392
Name: ID_IMDB, Length: 5241, dtype: object

In [8]:
lista_id_accion=id_accion.tolist()
lista_id_accion

['tt0411951',
 'tt0423474',
 'tt0429493',
 'tt0439801',
 'tt0446029',
 'tt0464162',
 'tt0473075',
 'tt0477080',
 'tt0480255',
 'tt0800320',
 'tt0810815',
 'tt0846004',
 'tt0861747',
 'tt0892769',
 'tt0903036',
 'tt0914823',
 'tt0923653',
 'tt0934981',
 'tt0938283',
 'tt0944835',
 'tt0947810',
 'tt0955308',
 'tt0960827',
 'tt0963966',
 'tt0964517',
 'tt0969653',
 'tt0973809',
 'tt0975738',
 'tt0985694',
 'tt0994917',
 'tt10010312',
 'tt1001526',
 'tt10028634',
 'tt10061904',
 'tt1013743',
 'tt1013860',
 'tt1020558',
 'tt1027683',
 'tt1032751',
 'tt1037705',
 'tt1038686',
 'tt1038685',
 'tt1038919',
 'tt10394606',
 'tt10392698',
 'tt1041804',
 'tt1053424',
 'tt10588452',
 'tt10622122',
 'tt10751652',
 'tt1075642',
 'tt1075747',
 'tt10761796',
 'tt1078597',
 'tt1078583',
 'tt1080019',
 'tt1082075',
 'tt1082577',
 'tt1084717',
 'tt1090753',
 'tt10919396',
 'tt10926954',
 'tt1093906',
 'tt1286146',
 'tt1103153',
 'tt1104001',
 'tt1107855',
 'tt1114731',
 'tt1116184',
 'tt1123373',
 'tt11261

In [30]:
# última peli en la que se quedó antes de parar el bucle

lista_id_accion.index("tt12151820")
lista_id_accion.index("tt12163662")

#tt12158090 en el navegador es la que ha dado el error 404, el siguiente es el primero que ha dado error de no encontrar el buscador -> tt12163662")
lista_id_accion.index("tt12158090")

189

In [26]:
# Llamo al navegador

driver=webdriver.Chrome()
driver.get("https://www.imdb.com/?ref_=nv_home")
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# Hago grande el navegador para que aparezcan todas las opciones posibles de la web

driver.maximize_window()

sleep(5) 

# Lista de "id" por la que quiero iterar dentro de la funcion

######lista_id_muestra= ['tt10579986','tt10597228','tt11097384','tt11152168','tt11237950'] 

######lista_muestraaccion=['tt0411951','tt0423474','tt0429493','tt0439801','tt0446029','tt0464162','tt0473075','tt0477080'] 

# Creo un archivo csv para agregar la informacion

csv_info_pelis = 'resultados_info_accion.csv'

# Llamo a la funcion 

resultados_muestra=llamar_info_peli(lista_id_accion,driver,csv_info_pelis)

Error al procesar la búsqueda para el ID tt10010312: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]"}
  (Session info: chrome=124.0.6367.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF675FF1572+60802]
	(No symbol) [0x00007FF675F6AC72]
	(No symbol) [0x00007FF675E27CE4]
	(No symbol) [0x00007FF675E76D4D]
	(No symbol) [0x00007FF675E76E1C]
	(No symbol) [0x00007FF675EBCE37]
	(No symbol) [0x00007FF675E9ABBF]
	(No symbol) [0x00007FF675EBA224]
	(No symbol) [0x00007FF675E9A923]
	(No symbol) [0x00007FF675E68FEC]
	(No symbol) [0x00007FF675E69C21]
	GetHandleVerifier [0x00007FF6762F420D+3217949]
	GetHandleVerifier [0x00007FF6763361A7+3488183]
	GetHandleVerifier [0x00007FF67632F12F+3459391]
	GetHandleVerifier [0x00007

KeyboardInterrupt: 

In [21]:
resultados_muestra

[('Tekken',
  'Dwight H. Little',
  'Alan B. Mc Elroy',
  'Jin Kazama es testigo de la muerte de su madre Jun por parte de Tekken.',
  '1h 31min',
  '4.8'),
 ('Venus & Vegas',
  'Demian Lichtenstein',
  'Eddie Guerra',
  'When three Vegas buddies attempt the score of a lifetime, they have to walk a fine line between their girlfriends who want their heads, and the mobsters who want them dead.',
  '1h 35min',
  '4.0'),
 ('El equipo A',
  'Joe Carnahan',
  'Joe Carnahan Brian Bloom Skip Woods',
  'Un grupo de veteranos de la guerra de Irak busca limpiar su nombre ante el ejército estadounidense, que sospecha que los cuatro hombres han cometido un crimen por el que fueron incriminados.',
  '1h 57min',
  '6.7'),
 ('Segurança Nacional',
  'Roberto Carminati',
  'Roberto Carminati Bruno Fantini Daniel Ortiz',
  "The Brazilian Government leads Latin American countries in the fight against drug cartels. In response drug cartels start a series of attacks against Brazil's cities and natural resou